In [86]:
from bs4 import BeautifulSoup
import requests
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import wordnet as wn
from collections import Counter
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [87]:
search_word = 'samsung'
title_list = []

In [88]:
def get_titles(start_num, end_num):
    # start_num ~ end_num까지 크롤링
    while 1:
        if start_num > end_num:
            break
        print(start_num)
        url = 'https://search.naver.com/search.naver?where=news&sm=tab_jum&query={}&start={}'.format(
            search_word, start_num)
        req = requests.get(url)
        # 정상적인 request 확인
        if req.ok:
            html = req.text
            soup = BeautifulSoup(html, 'html.parser')

            # 뉴스제목 뽑아오기
            titles = soup.select("ul > li > .news_wrap.api_ani_send > div > a")
            print(titles)

            # list에 넣어준다
            for title in titles:
                title_list.append(title['title'])
        start_num += 10
    print(title_list)

In [94]:
def make_wordcloud(word_count):
    twitter = TweetTokenizer()

    sentences_tag = []
    # 형태소 분석하여 리스트에 넣기
    for sentence in title_list:
        morph = twitter.tokenize(sentence)
        sentences_tag.append(morph)
        print(morph)
        print('-' * 30)

    print(sentences_tag)
    print('\n' * 3)

    noun_adj_list = []
    # 명사와 형용사만 구분하여 리스트에 넣기
    for sentence1 in sentences_tag:
        for word, tag in sentence1:
            if tag in ['NN','JJ']:
            # if tag in (wn.NOUN,wn.JJ):
                noun_adj_list.append(word)

    # 형태소별 count
    counts = Counter(noun_adj_list)
    tags = counts.most_common(word_count)
    print(tags)

    # wordCloud생성
    # 한글꺠지는 문제 해결하기위해 font_path 지정
    wc = WordCloud(font_path='/Library/Fonts/NanumGothic.ttf',
                   background_color='white', width=800, height=600)
    print(dict(tags))
    cloud = wc.generate_from_frequencies(dict(tags))
    plt.figure(figsize=(10, 8))
    plt.axis('off')
    plt.imshow(cloud)
    plt.show()

In [95]:
if __name__ == '__main__':
    # 1~200번게시글 까지 크롤링
    get_titles(1, 10)
    # 단어 30개까지 wordcloud로 출력
    make_wordcloud(10)

1
[<a class="news_tit" href="http://www.koreatimes.co.kr/www/tech/2020/12/133_301460.html" onclick="return goOtherCR(this, 'a=nws*f.tit&amp;r=1&amp;i=8818858c_000000000000000000190530&amp;g=5668.0000190530&amp;u='+urlencode(this.href));" target="_blank" title="Samsung to enable S Pen for 2021 Galaxy S"><mark>Samsung</mark> to enable S Pen for 2021 Galaxy S</a>, <a class="news_tit" href="http://www.koreatimes.co.kr/www/culture/2020/12/199_301455.html" onclick="return goOtherCR(this, 'a=nws*f.tit&amp;r=2&amp;i=8818858c_000000000000000000190531&amp;g=5668.0000190531&amp;u='+urlencode(this.href));" target="_blank" title="R.E.S.C.U.E: Samsung Fashion Institute predicts next year's trend">R.E.S.C.U.E: <mark>Samsung</mark> Fashion Institute predicts next year's trend</a>, <a class="news_tit" href="http://www.koreatimes.co.kr/www/tech/2020/12/133_301414.html" onclick="return goOtherCR(this, 'a=nws*f.tit&amp;r=3&amp;i=8818858c_000000000000000000190501&amp;g=5668.0000190501&amp;u='+urlencode(thi

ValueError: too many values to unpack (expected 2)